In [1]:
import os
import re
import bs4
import json
import time
import lxml
import deepl
import requests
import cloudscraper
from pathlib import Path
from googletrans import Translator
from fake_useragent import UserAgent
from IPython.display import clear_output as clear

In [2]:
ua = UserAgent()
translator = Translator()
dl_translator = deepl.Translator("4a2f6ee1-0d7a-8c77-9b1d-9476a086f204:fx")

In [3]:
headers = {"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           "accept-encoding": "gzip, deflate, br",
           "accept-language": "en-GB,en;q=0.9,es-ES;q=0.8,es;q=0.7,en-US;q=0.6,eu;q=0.5",
           "cache-control": "max-age=0",
           "cookie": "t=238707487; _ga=GA1.2.1376835774.1641262578; _gid=GA1.2.1482423077.1641262578; _fbp=fb.1.1641262579526.851471446",
           "referer": "https://hidemy.name/en/proxy-list/?start=64",
           "sec-ch-ua-mobile": "?0",
           "sec-ch-ua-platform": "macOS",
           "sec-fetch-dest": "document",
           "sec-fetch-mode": "navigate",
           "sec-fetch-site": "same-origin",
           "sec-fetch-user": "?1",
           "upgrade-insecure-requests": "1",
           "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"}

In [4]:
language = 'german'

In [5]:
verbs = [
    "werden",
    "sollen",
    "führen",
    "bleiben",
    "erkennen",
    "halten",
    "bestehen",
    "ergeben",
    "freien",
    "nennen",
    "suchen",
    "erfahren",
    "beziehen",
    "treffen",
    "darstellen",
    "erfolgen",
    "erfüllen",
    "ließen",
    "bezeichnen",
    "besitzen",
    "glauben",
    "handeln",
    "befinden",
    "feststellen",
    "sichern",
    "erfassen",
    "vermitteln",
    "verfügen",
    "legen",
    "behaupten",
    "mögen",
    "gestalten",
    "richten",
    "wählen",
    "verbinden",
    "begründen",
    "beachten",
    "zählen",
    "fassen",
    "vorstellen",
    "übersehen",
    "sorgen",
    "erweisen",
    "belegen",
    "gewährleisten",
    "schönen",
    "erzielen",
    "durchsetzen",
    "verhalten",
    "eingehen",
    "stützen",
    "bewahren",
    "entnehmen",
    "betonen",
    "verschaffen",
    "einsetzen",
    "eintreten",
    "verdeutlichen",
    "bewirken",
    "ergänzen",
    "leiten",
    "genügen",
    "schlagen",
    "entfalten",
    "beseitigen",
    "ermitteln",
    "aufnehmen",
    "entlassen",
    "greifen",
    "beteiligen",
    "klären",
    "ausüben",
    "erweitern",
    "umfassen",
    "erleichtern",
    "bewältigen",
    "bedenken",
    "vorkommen",
    "bauen",
    "durchführen",
    "tätigen",
    "ausmachen",
    "suchten",
    "teilnehmen",
    "erschließen",
    "festhalten",
    "spüren",
    "errichten",
    "unterhalten",
    "rufen",
    "beherrschen",
    "stoßen",
    "pflegen",
    "sammeln",
    "aufbauen",
    "kennzeichnen",
    "widmen",
    "antworten",
    "einstellen",
    "grünen",
    "angeben",
    "veranlassen",
    "erarbeiten",
    "schätzen",
    "abgeben",
    "beschaffen",
    "lenken",
    "schweigen",
    "zustimmen",
    "festlegen",
    "kümmern",
    "drängen",
    "begrüßen",
    "verwandeln",
    "profitieren",
    "zusammenfassen",
    "beschließen",
    "erteilen",
    "prägen",
    "stecken",
    "basieren",
    "anzubieten",
    "bearbeiten",
    "übergehen",
    "unterstellen",
    "wecken",
    "verfallen",
    "aufzuheben",
    "herrschen",
    "übereinstimmen",
    "unterwerfen",
    "stillen",
    "verraten",
    "ausfallen",
    "zurückführen",
    "drücken",
    "anpassen",
    "kosten",
    "ausführen",
    "erschweren",
    "auseinandersetzen",
    "versetzen",
    "aussprechen",
    "füllen",
    "mitteilen",
    "verbieten",
    "einbringen",
    "bescheiden",
    "festigen",
    "belasten",
    "abzulehnen",
    "verstoßen",
    "aufhalten",
    "fügen",
    "verdrängen",
    "erledigen",
    "unterlassen",
    "aufheben",
    "vergrößern",
    "erziehen",
    "ahnen",
    "sicherstellen",
    "verknüpfen",
    "beklagen",
    "ansprechen",
    "besprechen",
    "behindern",
    "unterlaufen",
    "scheiden",
    "knüpfen",
    "ansetzen",
    "überwiegen",
    "zerschlagen",
    "sprengen",
    "drehen",
    "zusammenarbeiten",
    "weitergehen",
    "einberufen",
    "herausstellen",
    "unterdrücken",
    "erblicken",
    "nachgehen",
    "ablaufen",
    "bezeugen",
    "befördern",
    "abnehmen",
    "aufbringen",
    "zurückziehen",
    "bejahen",
    "durchbrechen",
    "beheben",
    "räumen",
    "verabschieden",
    "täuschen",
    "mitwirken",
    "erringen",
    "unterbinden",
    "vermehren",
    "hervorheben",
    "werben",
    "haften",
    "verkünden",
    "hüten",
    "teilhaben",
    "auskommen",
    "übermitteln",
    "anlegen",
    "hegen",
    "befragen",
    "auswählen",
    "antreten",
    "erfreuen",
    "vernehmen",
    "ausgleichen",
    "verbanden",
    "reifen",
    "praktizieren",
    "vergegenwärtigen",
    "vertreiben",
    "verschärfen",
    "angehen",
    "einhergehen",
    "wiederfinden",
    "manifestieren",
    "schwanken",
    "verschließen",
    "andeuten",
    "nachkommen",
    "heranziehen",
    "erstatten",
    "zuschreiben",
    "schöpfen",
    "entbehren",
    "verschmelzen",
    "verleiten",
    "einleiten",
    "abbilden",
    "abgrenzen",
    "betätigen",
    "bergen",
    "verlagern",
    "unterlegen",
    "distanzieren",
    "ausdehnen",
    "aufgreifen",
    "versammeln",
    "blockieren",
    "zusammenfallen",
    "raten",
    "flüchten",
    "abbauen",
    "aushalten",
    "absetzen",
    "beschreiten",
    "entgegentreten",
    "eindringen",
    "mitmachen",
    "verdecken",
    "belehren",
    "vertragen",
    "konzipieren",
    "profilieren",
    "vorwerfen",
    "verschweigen",
    "untergehen",
    "aufsteigen",
    "irren",
    "verbuchen",
    "unterbleiben",
    "anrufen",
    "befallen",
    "ausnutzen",
    "erschlagen",
    "erahnen",
    "kommentieren",
    "überfallen",
    "überlagern",
    "einschlagen",
    "anordnen",
    "widerfahren",
    "zeitigen",
    "vorantreiben",
    "anziehen",
    "ausbleiben",
    "herausarbeiten",
    "erwirtschaften",
    "aufbrechen",
    "überstehen",
    "aufklären",
    "zerlegen",
    "inszenieren",
    "sicherten",
    "ragen",
    "erlegen",
    "ausweiten",
    "zugreifen",
    "wiegen",
    "anmelden",
    "mitbestimmen",
    "absichern",
    "entkräften",
    "hingeben",
    "nachgeben",
    "ablenken",
    "umschlagen",
    "bannen",
    "umkehren",
    "herhalten",
    "weigern",
    "ausschalten",
    "einziehen",
    "lindern",
    "vorbringen",
    "unterschlagen",
    "verderben",
    "festmachen",
    "vorführen",
    "zurückbleiben",
    "aufpassen",
    "vergrößerten",
    "durchhalten",
    "abstellen",
    "abziehen",
    "ernten",
    "abtreten",
    "vorangehen",
    "herrühren",
    "befestigen",
    "unterbringen",
    "aufteilen",
    "auslegen",
    "bestärken",
    "abzeichnen",
    "spannen",
    "verehren",
    "durchziehen",
    "zusammentreffen",
    "obliegen",
    "fortbestehen",
    "beharren",
    "erschrecken",
    "fesseln",
    "zusammenwirken",
    "hingehen",
    "aufwenden",
    "zurechnen",
    "gebären",
    "instrumentalisieren",
    "beschneiden",
    "einschalten",
    "beschweren",
    "erden",
    "abwehren",
    "anbringen",
    "zusammentreten",
    "ausstellen",
    "aufdrängen",
    "durchschlagen",
    "zittern",
    "verfestigen",
    "assoziieren",
    "aussteigen",
    "hinreißen",
    "offenstehen",
    "durchblicken",
    "einsteigen",
    "leihen",
    "abzufinden",
    "entladen",
    "büßen",
    "vorlesen",
    "entschädigen",
    "anstehen",
    "kargen",
    "verkümmern",
    "ausziehen",
    "weißen",
    "kandidieren",
    "bewirtschaften",
    "erübrigen",
    "darbieten",
    "entzünden",
    "erstaunen",
    "bekleiden",
    "rütteln",
    "verziehen",
    "rangieren",
    "umstellen",
    "abschrecken",
    "vervielfältigen",
    "antun",
    "ausscheiden",
    "strahlen",
    "herangehen",
    "verwehren",
    "verlauten",
    "kapitulieren",
    "ausnehmen",
    "erklingen",
    "hungern",
    "niederlassen",
    "freimachen",
    "überdecken",
    "miterleben",
    "zusammenfügen",
    "sanieren",
    "verlangsamen",
    "streifen",
    "mittragen",
    "umarmen",
    "abschwächen",
    "weiterhelfen",
    "herkommen",
    "beilegen",
    "aufrichten",
    "gönnen",
    "lähmen",
    "zurückreichen",
    "tadeln",
    "stürmen",
    "erkaufen",
    "mißachten",
    "benachrichtigen",
    "mitspielen",
    "entbinden",
    "durchkreuzen",
    "heraufbeschwören",
    "abreißen",
    "einbüßen",
    "spezialisieren",
    "austreten",
    "verfangen",
    "einmünden",
    "zurückgewinnen",
    "anleiten",
    "besteigen",
    "durchmachen",
    "schimpfen",
    "aufwarten",
    "ahnden",
    "funktionierten",
    "beleuchteten",
    "auslöschen",
    "vertauschen",
    "angleichen",
    "nachlassen",
    "absinken",
    "überziehen",
    "tollen",
    "prallen",
    "klarstellen",
    "durchleuchten",
    "weiterleiten",
    "herankommen",
    "genesen",
    "verrechnen",
    "loslassen",
    "missen",
    "honorieren",
    "herausholen",
    "benachteiligen",
    "vorstoßen",
    "verfeinern",
    "rezipieren",
    "festschreiben",
    "herantreten",
    "besänftigen",
    "ausleben",
    "ausdenken",
    "triumphieren",
    "ausklammern",
    "eindämmen",
    "umgestalten",
    "übergreifen",
    "pochen",
    "zurückdrängen",
    "züchten",
    "voranbringen",
    "abschreiben",
    "rannten",
    "entwachsen",
    "vermieten",
    "heulen",
    "animieren",
    "stopfen",
    "beziffern",
    "beschimpfen",
    "wähnen",
    "aufarbeiten",
    "schleifen",
    "ausstehen",
    "beiwohnen",
    "einreihen",
    "zersetzen",
    "vorhaben",
    "zurückzahlen",
    "einstecken",
    "überfahren",
    "gutheißen",
    "ankreuzen"
]

In [6]:
def getCollinsTranslations(verb, language):

    translations = []

    try:
        url = f'https://www.collinsdictionary.com/dictionary/{language}-english/{verb}'
        scraper = cloudscraper.create_scraper(
            browser={
                'browser': 'firefox',
                'platform': 'windows',
                'mobile': False
            })
        soup = bs4.BeautifulSoup(scraper.get(url, headers = {"useragent": f"{ua.random}"}).text, 'lxml')

        if 'Cloudflare' not in soup.select('title')[0].getText():

            for translation in soup.select('.page .dictionary .sense:not(.type-example)>.cit.type-translation .quote'):
                if translation.getText().strip()[:3] == 'to ':
                    translations.append(translation.getText().strip())

    except:
        pass


    return translations

In [7]:
def getWiktionaryTranslations(verb, language):
    translations = []

    page = requests.get(f'https://en.wiktionary.org/wiki/{verb}#{language.capitalize()}')
    section = page.text.split(f'<h2><span class="mw-headline" id="{language.capitalize()}">')[1].split('<h2>')[0]
    soup = bs4.BeautifulSoup(section, 'lxml')

    for ol in soup.select('ol'):
        for li in ol.select('li'):
            translation = li.getText().split(') ')[1].split(' (')[0].split('\n')[0]
            translations.append(translation)

    return translations

In [8]:
def getWordReferenceTranslation(verb, language):
    translations = []

    lang = {'spanish': 'spen', 'french': 'fren', 'german': 'deen', 'italian': 'iten', 'portuguese': 'pten'}
    page = requests.get(f'https://www.wordreference.com/{lang[language]}/{verb}')
    soup = bs4.BeautifulSoup(page.text, 'lxml')

    for section in soup.select('.WRD'):

        append = False

        for row in section.find_all("tr", class_ = ['even','odd']):
            
            if row.select('.ToWrd'):
            
                if row.select('.FrWrd'):
                    origin = row.select('.FrWrd')[0]
                    
                    if origin.select('strong')[0].getText() == verb and origin.select('em')[0].getText().lower()[0] == 'v':
                        append = True
                    else:
                        append = False
                                
                if append and row.select('.ToWrd'):
                    raw = row.select('.ToWrd')[0].getText()

                    if row.select('.ToWrd')[0].select('em')[0].getText()[0] == 'v':
                        translation = raw.split('⇒')[0].split(' v ')[0].split(' vi ')[0].split(' vtr ')[0]
                        translations.append(translation)

    return translations

In [9]:
translations = {}

for i in range(0, len(verbs)):

    verb = verbs[i]

    translations[verb] = {}

    try:
        translations[verb]['collins'] = getCollinsTranslations(verb, language)[:5]
    except:
        pass
    
    try:
        translations[verb]['wiktionary'] = getWiktionaryTranslations(verb, language)[:5]
    except:
        pass

    try:
        translations[verb]['wordreference'] = getWordReferenceTranslation(verb, language)[:5]
    except:
        pass

    clear(); print(f'{verb} ({round(((i + 1)/len(verbs)) * 100, 3)}%)')

ankreuzen (100.0%)


In [10]:
translations

{'werden': {'collins': ['to get', 'to become', 'to turn into'],
  'wiktionary': ['will; to be going',
   'to be done; forms the passive voice',
   'to become; to get; to grow; to turn',
   'to begin or come to feel or experience',
   'to be, to happen, to occur'],
  'wordreference': ['get', 'become', 'get', 'become', 'train [sb]  as']},
 'sollen': {'collins': [],
  'wiktionary': ['should; to be obligated',
   'to be recommended',
   'to be intended',
   'to be said',
   'would; indicates that the subordinate clause indicates something that would happen in the past but after the time frame of the main clause'],
  'wordreference': ['shall', 'ought to']},
 'führen': {'collins': ['to take',
   'to lead',
   'to run',
   'to lead, to head',
   'to captain'],
  'wordreference': ['manage', 'lead', 'lead to', 'result in', 'keep']},
 'bleiben': {'collins': ['to stay, to remain',
   'to stay',
   'to stay, to remain',
   'to be left, to remain'],
  'wordreference': ['stay', 'remain', 'stay', 'ho